# Tech Layoffs Data Cleaning in SQL

I have copied this notework **[Tech Layoffs Data Cleaning in SQL] (https://www.kaggle.com/code/wilfridawere/tech-layoffs-data-cleaning-in-sql)** and take it as a starting point for a SQL exercise. The original notebook used *SQL (sqlite3)* to clean a dataset I put on Kaggle **[Tech layoffs 2020 - 2024](https://www.kaggle.com/datasets/ulrikeherold/tech-layoffs-2020-2024)** but a updated version. 

In [1]:
import numpy as np
import pandas as pd
import sqlite3 # for using SQL commands

In [2]:
# Load the data
tech_layoffs = pd.read_csv('/kaggle/input/tech-layoffs-2020-2024/tech_layoffs_Q2_2024.csv')
df = tech_layoffs.copy() # make a copy
df.head()

#        Company    Location_HQ                  Region       State  \
0  1  Tamara Mellon    Los Angeles                     NaN  California   
1  2   HopSkipDrive    Los Angeles                     NaN  California   
2  3    Panda Squad  San Francisco  San Francisco Bay Area  California   
3  4       Help.com         Austin                     NaN         NaN   
4  5      Inspirato         Denver                     NaN         NaN   

  Country      Continent  Laid_Off Date_layoffs Percentage  \
0     USA  North America      20.0   2020-03-12       40,0   
1     USA  North America       8.0   2020-03-13       10,0   
2     USA  North America       6.0   2020-03-13       75,0   
3     USA  North America      16.0   2020-03-16      100,0   
4     USA  North America     130.0   2020-03-16       22,0   

  Company_Size_before_Layoffs Company_Size_after_layoffs        Industry  \
0                         50                         30           Retail   
1                         80                         72   Transportation   
2                          8                          2         Consumer   
3                         16                          0          Support   
4                        591                        461           Travel   

      Stage  Money_Raised_in__mil  Year   latitude   longitude  
0  Series C                  90.0  2020  34.053691 -118.242766  
1   Unknown                  45.0  2020  34.053691 -118.242766  
2      Seed                   1.0  2020  37.779259 -122.419329  
3      Seed                   6.0  2020  30.271129  -97.743700  
4  Series C                  79.0  2020  39.739236 -104.984862

In [3]:
df.info()  # 1839 rows and 18 columns

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1839 entries, 0 to 1838
Data columns (total 18 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   #                            1839 non-null   int64  
 1   Company                      1839 non-null   object 
 2   Location_HQ                  1839 non-null   object 
 3   Region                       473 non-null    object 
 4   State                        566 non-null    object 
 5   Country                      1839 non-null   object 
 6   Continent                    1839 non-null   object 
 7   Laid_Off                     1677 non-null   float64
 8   Date_layoffs                 1839 non-null   object 
 9   Percentage                   1667 non-null   object 
 10  Company_Size_before_Layoffs  1585 non-null   object 
 11  Company_Size_after_layoffs   1619 non-null   object 
 12  Industry                     1839 non-null   object 
 13  Stage             

The columns `Laid_Off`, `Percentage`, `Company_Size_before_Layoffs`, `Company_Size_after_layoffs` have missing values so I will inspect them. This helps decide whether to impute or drop those rows with missing values. I don't need the `Money_Raised_in_$_mil` column

# Connect to a Database and convert the Dataframe to a Table

In [4]:
# Connect to the database
conn = sqlite3.connect('tech_layoffs_data.db')

In [5]:
# Create 'tech_layoffs' table
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='tech_layoffs'")
table_exists = len(cursor.fetchall()) > 0

if not table_exists:
  # Load data to 'tech_layoffs' table only if it doesn't exist
  df.to_sql('tech_layoffs', conn, index=False)
  print("Table 'tech_layoffs' created and data loaded successfully.")
else:
  print("Table 'tech_layoffs' already exists. Skipping data loading.")

Table 'tech_layoffs' created and data loaded successfully.


In [6]:
# Now I will use SQL commands to Clean data from the 'tech_layoffs' table
# pd.read_sql_query is an optimized approach for working with pandas and sqlite3

# Inspect the first_10_rows
first_10_rows = pd.read_sql_query("""SELECT *
                                   FROM tech_layoffs
                                   LIMIT 10;""", conn)
first_10_rows

#          Company    Location_HQ                  Region       State  \
0   1    Tamara Mellon    Los Angeles                    None  California   
1   2     HopSkipDrive    Los Angeles                    None  California   
2   3      Panda Squad  San Francisco  San Francisco Bay Area  California   
3   4         Help.com         Austin                    None        None   
4   5        Inspirato         Denver                    None        None   
5   6        Flytedesk        Boulder                    None        None   
6   7      Remote Year        Chicago                    None        None   
7   8           CTO.ai      Vancouver                    None        None   
8   9  Flywheel Sports  New York City                    None        None   
9  10          Compass  New York City                    None        None   

  Country      Continent  Laid_Off Date_layoffs Percentage  \
0     USA  North America      20.0   2020-03-12       40,0   
1     USA  North America       8.0   2020-03-13       10,0   
2     USA  North America       6.0   2020-03-13       75,0   
3     USA  North America      16.0   2020-03-16      100,0   
4     USA  North America     130.0   2020-03-16       22,0   
5     USA  North America       4.0   2020-03-18       20,0   
6     USA  North America      50.0   2020-03-19       50,0   
7  Canada  North America      30.0   2020-03-20       50,0   
8     USA  North America     784.0   2020-03-20       98,0   
9     USA  North America     375.0   2020-03-23       15,0   

  Company_Size_before_Layoffs Company_Size_after_layoffs        Industry  \
0                         50                         30           Retail   
1                         80                         72   Transportation   
2                          8                          2         Consumer   
3                         16                          0          Support   
4                        591                        461           Travel   
5                         20                         16        Marketing   
6                        100                         50           Travel   
7                         60                         30   Infrastructure   
8                        800                         16          Fitness   
9                       2500                       2125      Real Estate   

      Stage  Money_Raised_in__mil  Year   latitude   longitude  
0  Series C                  90.0  2020  34.053691 -118.242766  
1   Unknown                  45.0  2020  34.053691 -118.242766  
2      Seed                   1.0  2020  37.779259 -122.419329  
3      Seed                   6.0  2020  30.271129  -97.743700  
4  Series C                  79.0  2020  39.739236 -104.984862  
5      Seed                   4.0  2020  40.014986 -105.270545  
6  Series B                  17.0  2020  41.875562  -87.624421  
7      Seed                   7.0  2020  49.260872 -123.113952  
8  Acquired                 120.0  2020  40.712728  -74.006015  
9  Series G                1600.0  2020  40.712728  -74.006015

# Data Cleaning

# Identify Spelling Errors

In [7]:
# Inspect the Industry column 
industry = pd.read_sql_query("""
SELECT DISTINCT Industry
FROM tech_layoffs;
""",conn)

industry

Industry
0                             Retail
1                     Transportation
2                           Consumer
3                            Support
4                             Travel
5                          Marketing
6                     Infrastructure
7                            Fitness
8                        Real Estate
9                               Data
10                           Finance
11                                HR
12                              Food
13                           Product
14                             Media
15                         Education
16                             Other
17                            Energy
18                         Aerospace
19                         Logistics
20                          Security
21                        Healthcare
22                      Construction
23                             Legal
24                            Crypto
25                             Sales
26                          Hardware
27                     Manufacturing
28              Software Development
29                        e-commerce
30                                AI
31                    Semiconductors
32           Entertainment Providers
33                Telecommunications
34                           Fintech
35                       IT Services
36                            Gaming
37  Business Consulting and Services
38       Motor Vehicle Manufacturing
39                        Appliances
40               Social posting site
41                         Insurance
42                       Hospitality
43            Biotechnology Research
44                           E-sport
45                              SaaS
46      Translation and Localization
47                             cloud
48            Environmental Services

# Duplicate Rows

In [8]:
# Identify rows that have duplicate values in all these columns. I will drop the duplicate rows later
# Company, Location_HQ, Country, Laid_Off, Date_layoffs, Industry

duplicate_rows = pd.read_sql_query("""
SELECT *
FROM (
  SELECT Company, Location_HQ, Country, Laid_Off, Date_layoffs, Industry,
         ROW_NUMBER() OVER (PARTITION BY Company, Location_HQ, Country, Laid_Off, Date_layoffs, Industry) AS row_num
  FROM tech_layoffs
) AS duplicates  
WHERE row_num > 1;
""", conn)

duplicate_rows

Company    Location_HQ Country  Laid_Off Date_layoffs   Industry  row_num
0  Criteo          Paris  France     140.0   2024-04-12  Marketing        2
1    Etsy  New York City     USA     225.0   2023-12-13     Retail        2
2    Tome  San Francisco     USA      12.0   2024-04-16         AI        2

In [9]:
# Inspect any of the duplicate rows
Criteo = pd.read_sql_query("""
SELECT Company, Location_HQ, Country, Laid_Off,Date_layoffs, Industry
FROM tech_layoffs
WHERE Company LIKE 'Criteo';
""",conn)

Criteo

Company Location_HQ Country  Laid_Off Date_layoffs   Industry
0  Criteo       Paris  France     140.0   2024-04-12  Marketing
1  Criteo       Paris  France     140.0   2024-04-12  Marketing

In [10]:
# Inspect another duplicate row
Etsy = pd.read_sql_query("""
SELECT Company, Location_HQ, Country, Laid_Off,Date_layoffs, Industry
FROM tech_layoffs
WHERE Company LIKE 'Etsy'
""",conn)

Etsy

Company    Location_HQ Country  Laid_Off Date_layoffs Industry
0    Etsy  New York City     USA     225.0   2023-12-13   Retail
1    Etsy  New York City     USA     225.0   2023-12-13   Retail

In [11]:
# Inspect another duplicate row
Tome = pd.read_sql_query("""
SELECT Company, Location_HQ, Country, Laid_Off,Date_layoffs, Industry
FROM tech_layoffs
WHERE Company LIKE 'Tome'
""",conn)

Tome

Company    Location_HQ Country  Laid_Off Date_layoffs Industry
0    Tome  San Francisco     USA      12.0   2024-04-16       AI
1    Tome  San Francisco     USA      12.0   2024-04-16       AI

# Missing Values

In [12]:
# Missing values in Laid_Off, Percentage, Company_Size_before_Layoffs, Company_Size_after_layoffs columns
missing_values = pd.read_sql_query("""
SELECT Laid_Off, Percentage, Company_Size_before_Layoffs, Company_Size_after_layoffs
FROM tech_layoffs
WHERE Laid_Off IS NULL
OR Percentage IS NULL 
OR Company_Size_before_Layoffs IS NULL 
OR Company_Size_after_layoffs IS NULL;
""",conn)

for col in missing_values.columns:
    print(f"Missing values in '{col}': {missing_values[col].isnull().sum()}")

print() # blank line
missing_values.head(10) # the first 10 rows

Missing values in 'Laid_Off': 162
Missing values in 'Percentage': 172
Missing values in 'Company_Size_before_Layoffs': 254
Missing values in 'Company_Size_after_layoffs': 220



Laid_Off Percentage Company_Size_before_Layoffs Company_Size_after_layoffs
0       NaN       None                        None                       None
1       NaN      100,0                        None                         0 
2       NaN      100,0                        None                         0 
3      45.0       None                        None                       None
4     111.0       None                        None                       None
5      31.0       None                        None                       None
6     109.0       None                        None                       None
7       NaN       15,0                        None                       None
8       NaN       20,0                        None                       None
9       NaN      100,0                        None                         0

# The Problem with Imputation

In this case, the columns `Laid_Off`, `Percentage`, `Company_Size_before_Layoffs`, and `Company_Size_after_layoffs` are all **related**.

Imagine these rows represent companies that went through layoffs.  Since we don't have all the information *(number laid off, company size before/after)*,  trying to guess those missing values would be like making up a story.  It's better to remove these rows and focus on the companies where we have complete data for a more accurate analysis.

In [13]:
# FILTER the data, EXCLUDING rows with missing values in the specified columns
Cleaned_tech_layoffs = pd.read_sql_query("""
SELECT *
FROM tech_layoffs
WHERE Laid_Off IS NOT NULL
AND Percentage IS NOT NULL 
AND Company_Size_before_Layoffs IS NOT NULL 
AND Company_Size_after_layoffs IS NOT NULL;
""",conn)

print(Cleaned_tech_layoffs.shape) # 1572 clean rows remain after Filtering 
print() # blank line
Cleaned_tech_layoffs.head()

(1572, 18)



#        Company    Location_HQ                  Region       State  \
0  1  Tamara Mellon    Los Angeles                    None  California   
1  2   HopSkipDrive    Los Angeles                    None  California   
2  3    Panda Squad  San Francisco  San Francisco Bay Area  California   
3  4       Help.com         Austin                    None        None   
4  5      Inspirato         Denver                    None        None   

  Country      Continent  Laid_Off Date_layoffs Percentage  \
0     USA  North America      20.0   2020-03-12       40,0   
1     USA  North America       8.0   2020-03-13       10,0   
2     USA  North America       6.0   2020-03-13       75,0   
3     USA  North America      16.0   2020-03-16      100,0   
4     USA  North America     130.0   2020-03-16       22,0   

  Company_Size_before_Layoffs Company_Size_after_layoffs        Industry  \
0                         50                         30           Retail   
1                         80                         72   Transportation   
2                          8                          2         Consumer   
3                         16                          0          Support   
4                        591                        461           Travel   

      Stage  Money_Raised_in__mil  Year   latitude   longitude  
0  Series C                  90.0  2020  34.053691 -118.242766  
1   Unknown                  45.0  2020  34.053691 -118.242766  
2      Seed                   1.0  2020  37.779259 -122.419329  
3      Seed                   6.0  2020  30.271129  -97.743700  
4  Series C                  79.0  2020  39.739236 -104.984862

In [14]:
# Drop DUPLICATE rows from the Dataframe, based on the columns used earlier when inspecting Duplicates
Cleaned_tech_layoffs = Cleaned_tech_layoffs.drop_duplicates(subset=['Company', 'Location_HQ', 'Country', 'Laid_Off',
                                                                    'Date_layoffs','Industry'], keep='first') # keep first instances

print(Cleaned_tech_layoffs.shape) # 1569 rows remain after dropping duplicate rows

(1569, 18)


In [15]:
# Save the cleaned DataFrame to a CSV file
Cleaned_tech_layoffs.to_csv('Cleaned_tech_layoffs.csv', index=False)
print('Cleaned_tech_layoffs.csv saved.')

Cleaned_tech_layoffs.csv saved.


In [16]:
# Close the connection
conn.commit()  # Save changes if any
conn.close()


***Working with Data, Explore Data, stay curious***